In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score

import os

import warnings
warnings.filterwarnings("ignore")

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.listdir("/content/drive/MyDrive/11785 Coursework/DL Proj/result/")

['fingpt_sentiment_result_batch_0_transcriptcomponent.pkl',
 'fingpt_sentiment_result_batch_1_transcriptcomponent.pkl',
 'batch_0_result.txt']

In [3]:
import os
DATA_DIR = "/content/drive/MyDrive/11785 Coursework/DL Proj/data/"
os.listdir(DATA_DIR)
metrics_result = pd.read_csv(DATA_DIR + "metrics.csv")[["transcriptid","label"]]

# fingpt result

In [4]:
result_dir = "/content/drive/MyDrive/11785 Coursework/DL Proj/result/"
data_dir = "/content/drive/MyDrive/11785 Coursework/DL Proj/data/"

In [5]:
# read text
import re
from collections import Counter

file_path = result_dir + "batch_0_result.txt"  # Replace with your actual file path

with open(file_path, "r", encoding="utf-8") as file:
    log_lines = file.readlines()

# Initialize counter
sentiment_counts = Counter()

# Process each line
for line in log_lines:
    # Extract the part after the last ']'
    match = re.split(r"\] ?", line.strip())
    if len(match) > 1:
        sentiment_part = match[-1].strip()  # Get text after last ']'
        words = sentiment_part.split()  # Split words
        for word in words:
            if word in ["positive", "negative", "neutral"]:
                sentiment_counts[word] += 1

# Display results
print("Sentiment Counts:")
print(sentiment_counts)

Sentiment Counts:
Counter({'positive': 463, 'neutral': 405, 'negative': 25})


In [6]:
import re

def process_sentiments_from_file(file_path):
    try:
        # Read the file
        with open(file_path, "r", encoding="utf-8") as file:
            log_lines = file.readlines()

        results = []

        # Process each line
        for line in log_lines:
            # Extract the part after the last ']'
            match = re.split(r"\] ?", line.strip())
            if len(match) > 1:
                sentiment_part = match[-1].strip()  # Get text after last ']'
                words = sentiment_part.split()  # Split words

                # Check for sentiment words
                for sentiment in ["positive", "negative", "neutral"]:
                    if sentiment in words:
                        results.append(sentiment)
                        break
                else:
                    results.append(None)  # If no sentiment found, append None
            else:
                results.append(None)  # If no valid sentiment part, append None

        return results

    except FileNotFoundError:
        print("Error: File not found.")
        return None

# Example usage
file_path = result_dir + "batch_0_result.txt"  # Replace with your actual file path
sentiment_results = process_sentiments_from_file(file_path)

print("Sentiment Results:", sentiment_results)

Sentiment Results: [None, None, 'positive', 'positive', 'positive', None, 'neutral', None, None, None, 'positive', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'neutral', None, None, None, None, None, None, None, None, None, None, None, None, 'neutral', 'neutral', None, None, None, None, None, None, None, None, 'neutral', None, None, None, None, None, None, None, None, None, None, None, None, 'neutral', None, None, None, 'neutral', None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'neutral', None, None, None, None, 'positive', None, 'positive', 'positive', None, None, None, 'positive', None, 'positive', None, 'positive', None, None, None, None, None, None, 'neutral', None, 'neutral', 'neutral', None, None, None, 'neutral', 'neutral', 'neutral', None, 'neutral', None, None, None, None, None, None, None, None, None, None, None, None, None, None, 'neutral', None, None, 'neutral', 'neutral', None, No

In [7]:
len(sentiment_results)

4792

In [8]:
DATA_DIR = "/content/drive/My Drive/11785 Coursework/DL Proj/data/"
df_batch_0 = pd.read_parquet(DATA_DIR + "batch_0_transcriptcomponent.parquet")

In [9]:
df_batch_0.columns

Index(['transcriptcomponentid', 'transcriptid', 'componentorder',
       'transcriptcomponenttypeid', 'transcriptpersonid', 'componenttext'],
      dtype='object')

In [10]:
list_id = df_batch_0['transcriptid'].drop_duplicates().tolist()[:len(sentiment_results)]
df_result_fingpt = pd.DataFrame({'transcriptid': list_id, 'sentiment': sentiment_results})
df_result_fingpt['sentiment'] = df_result_fingpt['sentiment'].replace({'positive': 1, 'negative': -1, 'neutral': 0})
df_result_fingpt['sentiment'] = df_result_fingpt['sentiment'].replace({np.nan: 0})
df_result_fingpt

,transcriptid,sentiment
0,131,0.0
1,132,0.0
2,133,1.0
3,134,1.0
4,1654078,1.0
...,...,...
4787,3044757,0.0
4788,3055319,0.0
4789,3056729,0.0
4790,3058421,0.0


In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


In [13]:
model_result = pd.merge(df_result_fingpt, metrics_result, on="transcriptid")


accuracy = accuracy_score(model_result["sentiment"], model_result["label"]) * 100
precision = precision_score(model_result["sentiment"], model_result["label"], average='weighted') * 100
recall = recall_score(model_result["sentiment"], model_result["label"], average='weighted') * 100
f1 = f1_score(model_result["sentiment"], model_result["label"], average='weighted') * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1 Score: {f1:.2f}%")

Accuracy: 48.25%
Precision: 81.30%
Recall: 48.25%
F1 Score: 59.63%


# dictionary-based result

In [14]:
os.listdir(data_dir)

['df_earnings.parquet',
 'batch_0_transcriptcomponent.parquet',
 'batch_1_transcriptcomponent.parquet',
 'batch_2_transcriptcomponent.parquet',
 'batch_3_transcriptcomponent.parquet',
 'batch_4_transcriptcomponent.parquet',
 'batch_5_transcriptcomponent.parquet',
 'batch_6_transcriptcomponent.parquet',
 'batch_7_transcriptcomponent.parquet',
 'batch_8_transcriptcomponent.parquet',
 'batch_9_transcriptcomponent.parquet',
 'batch_10_transcriptcomponent.parquet',
 'batch_11_transcriptcomponent.parquet',
 'batch_12_transcriptcomponent.parquet',
 'batch_13_transcriptcomponent.parquet',
 'batch_14_transcriptcomponent.parquet',
 'batch_15_transcriptcomponent.parquet',
 'batch_16_transcriptcomponent.parquet',
 'batch_17_transcriptcomponent.parquet',
 'batch_18_transcriptcomponent.parquet',
 'batch_19_transcriptcomponent.parquet',
 'batch_20_transcriptcomponent.parquet',
 'batch_21_transcriptcomponent.parquet',
 'batch_22_transcriptcomponent.parquet',
 'batch_23_transcriptcomponent.parquet',
 '

In [15]:
df_sentiment_batch_0 = pd.read_parquet(data_dir + "sentiment_score_batch_0_transcriptcomponent.parquet")
df_sentiment_batch_1 = pd.read_parquet(data_dir + "sentiment_score_batch_1_transcriptcomponent.parquet")

In [16]:
df_result_0 = df_sentiment_batch_0.groupby("transcriptid")['sentiment_score'].median().reset_index()
df_result_1 = df_sentiment_batch_1.groupby("transcriptid")['sentiment_score'].median().reset_index()

df_result_merge = pd.concat([df_result_0, df_result_1]).reset_index(drop=True)

In [17]:
df_result_merge['sentiment_score'].describe()

,sentiment_score
count,10000.000000
mean,0.640614
std,0.164337
min,0.000000
25%,0.523700
50%,0.659700
75%,0.769100
max,1.000000


In [18]:
df_result_merge['sentiment'] = np.where(
    df_result_merge['sentiment_score'] >= 0.05, 1,  # Positive sentiment
    np.where(df_result_merge['sentiment_score'] <= -0.05, -1, 0)  # Negative or Neutral sentiment
)

df_result_merge = df_result_merge[['transcriptid', 'sentiment']]

In [19]:
df_result_merge

,transcriptid,sentiment
0,1239,1
1,1240,1
2,1306,1
3,1307,1
4,1308,1
...,...,...
9995,3366238,1
9996,3367247,1
9997,3367398,1
9998,3367567,1


In [20]:
model_result = pd.merge(df_result_merge, metrics_result, on="transcriptid")
print(accuracy_score(model_result["sentiment"], model_result["label"]))

0.2531734132933533


In [21]:
accuracy = accuracy_score(model_result["sentiment"], model_result["label"]) * 100
precision = precision_score(model_result["sentiment"], model_result["label"], average='weighted') * 100
recall = recall_score(model_result["sentiment"], model_result["label"], average='weighted') * 100
f1 = f1_score(model_result["sentiment"], model_result["label"], average='weighted') * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Precision: {precision:.2f}%")
print(f"Recall: {recall:.2f}%")
print(f"F1 Score: {f1:.2f}%")

Accuracy: 25.32%
Precision: 99.84%
Recall: 25.32%
F1 Score: 40.24%
